In [ ]:
#pip install pandas-profiling

In [ ]:
#pip install -r ../requirements.txt

In [ ]:
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.model_selection import KFold,GridSearchCV,train_test_split,StratifiedKFold
import matplotlib.pyplot as plt
import lightgbm as lgbm
import optuna
from sklearn.metrics import log_loss,f1_score

In [ ]:
#from pandas_profiling import ProfileReport

#profile_train = ProfileReport(train, title="Pandas Profiling Report")
#profile_test = ProfileReport(test, title="Pandas Profiling Report")

#profile_train.to_file("train.html")
#profile_test.to_file("test.html")

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample = pd.read_csv('../data/sample_submission.csv')

In [ ]:
for column in train.select_dtypes(include='bool').columns:
    train[column] = train[column].astype(int)
    
for column in test.select_dtypes(include='bool').columns:
    test[column] = test[column].astype(int)

In [ ]:
train['contents_open_dt'] = pd.to_datetime(train['contents_open_dt'])
test['contents_open_dt'] = pd.to_datetime(test['contents_open_dt'])

train.sort_values(by='contents_open_dt',inplace=True)

In [ ]:
for df in [train,test]:
    df['dia'] = pd.DatetimeIndex(df['contents_open_dt']).day
    df['hora'] = pd.DatetimeIndex(df['contents_open_dt']).hour
    df['dayofweek'] = df['contents_open_dt'].dt.dayofweek

In [ ]:
for df in [train,test]:
    df.drop(['person_prefer_f',
             'person_prefer_g'
             #'h_m_match_yn',
             #'person_attribute_a'
             #'person_prefer_e',
             #'contents_attribute_c'
            ],axis=1,inplace=True)

In [ ]:
#train = pd.get_dummies(train, columns=['person_attribute_a_1', 'person_attribute_b','person_prefer_c','person_prefer_e','contents_attribute_i','contents_attribute_j_1','contents_attribute_c','contents_attribute_m','contents_attribute_e'])
#test = pd.get_dummies(test, columns=['person_attribute_a_1', 'person_attribute_b','person_prefer_c','person_prefer_e','contents_attribute_i','contents_attribute_j_1','contents_attribute_c','contents_attribute_m','contents_attribute_e'])

#train = pd.get_dummies(train, columns=['contents_attribute_d'])
#test = pd.get_dummies(test, columns=['contents_attribute_d'])

In [ ]:
y = train['target']
X = train.drop(['id','target','contents_open_dt'],axis = 1)
#X.columns = np.arange(X.shape[1])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False)

#X_train.columns = np.arange(X_train.shape[1])

cv_data = lgbm.Dataset(X, label = y)

train_ = train[pd.DatetimeIndex(train['contents_open_dt']).month != 11]
val = train[pd.DatetimeIndex(train['contents_open_dt']).month == 11]

y_train = train_['target']
X_train = train_.drop(['id','target','contents_open_dt'],axis = 1)

y_test = val['target']
X_test = val.drop(['id','target','contents_open_dt'],axis = 1)

In [ ]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.95, step=0.1),
        "num_leaves": trial.suggest_int("num_leaves", 5, 100, step=5),
        "max_depth": trial.suggest_int("max_depth", 1, 30, step=3),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 50, 1000, step=50),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1, 30, step=3),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.1, 0.99, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq",[1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 0.99, step=0.1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 2000, step=10),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=False)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    #return np.mean(f1_scores)
    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value : {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

In [ ]:
model = lgbm.LGBMClassifier(objective="binary", **study.best_params)
model.fit(
    X_train,
    y_train
)

pred = model.predict(X_test)
f1_score(y_test, pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
param = study.best_params
param['objective'] = "binary"
param['verbosity'] = 0

In [ ]:
#model = lgbm.train(param, cv_data)
model = lgbm.LGBMClassifier(objective="binary", **study.best_params)
model.fit(
    X,
    y
)

In [ ]:
pred = model.predict(test.drop(['id','contents_open_dt'],axis = 1))
#pred_1 = model.predict_proba(test.drop(['id','contents_open_dt'],axis = 1))

In [ ]:
pd.DataFrame(pred).value_counts()

In [ ]:
sample['target'] = pred
sample.to_csv('sub.csv',index=False)